In [ ]:
import pandas as pd
import numpy as np
import zeyrek
import nltk##lemmatization için lazım 
from jpype import JClass, getDefaultJVMPath, shutdownJVM, startJVM, JString
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.ensemble import RandomForestRegressor
import re
from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer
#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#from transformers import AutoTokenizer, AutoModel
#import torch
#from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer

In [ ]:
df1 = pd.read_csv("./dep_pars(1)", sep='\t')
df2 = pd.read_csv("./dep_pars(2)", sep='\t')
df3 = pd.read_csv("./dep_pars(3)", sep='\t')
df4 = pd.read_csv("./dep_pars(4)", sep='\t')
df5 = pd.read_csv("./dep_pars(5)", sep='\t')
frames = [df1, df2,df3,df4,df5]
data = pd.concat(frames,ignore_index=True)
data.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.2'], axis='columns', inplace=True)

In [ ]:
data

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,words2,sentence_length,position,result
0,1064.0,Nikah yerinizi nasıl seçersiniz?,Evlilik başvurusu nasıl yapılır?,-1,0,0,1,"['Nikah', 'yerinizi', 'nasıl', 'seçersiniz']",0.010610,0.333333,0.000000
1,1064.0,Kimler nikah şahidi seçilebilir?,Evlilik başvurusu nasıl yapılır?,-1,0,1,1,"['Kimler', 'nikah', 'şahidi', 'seçilebilir']",0.010610,0.250000,0.000000
2,1064.0,Nikahla ilgili merak ettiğiniz her şey burada…...,Evlilik başvurusu nasıl yapılır?,-1,0,0,0,"['Nikahla', 'ilgili', 'merak', 'ettiğiniz', 'h...",0.029178,0.200000,0.000000
3,1064.0,Müstakbel eşinizle birlikte yaşadığınız yerdek...,Evlilik başvurusu nasıl yapılır?,-1,1,1,0,"['Müstakbel', 'eşinizle', 'birlikte', 'yaşadığ...",0.023873,0.166667,0.000000
4,1064.0,"Ancak, ayrı illerde oturuyorsanız kendi illeri...",Evlilik başvurusu nasıl yapılır?,-1,0,1,0,"['Ancak', 'ayrı', 'illerde', 'oturuyorsanız', ...",0.031830,0.142857,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
11387,1697.0,"Galatasaray, Gaziantepspor maçının hazırlıklar...",Floryada Lucas Neill rüzgarı,-1,2,0,0,"['Galatasaray', 'Gaziantepspor', 'maçının', 'h...",0.042254,0.076923,0.000000
11388,1698.0,"Başbakan Recep Tayyip Erdoğan, özel bir haber ...",Erdoğan bir dava daha kazandı,-1,3,1,3,"['Başbakan', 'Recep', 'Tayyip', 'Erdoğan', 'öz...",0.200000,1.000000,0.090909
11389,1698.0,Ankara 3.,Erdoğan bir dava daha kazandı,-1,0,0,0,"['Ankara', '3']",0.013793,0.500000,0.000000
11390,1698.0,Asliye Hukuk Mahkemesi'nde görülen davanın kar...,Erdoğan bir dava daha kazandı,-1,5,1,2,"['Asliye', 'Hukuk', 'Mahkemesinde', 'görülen',...",0.117241,0.333333,0.000000


In [ ]:
translator = str.maketrans('', '', string.punctuation)
nltk.download('punkt')
stopword_remover = StopwordRemover()
analyzer = zeyrek.MorphAnalyzer()

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
model = AutoModel.from_pretrained('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
def giveEmbeddings(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embeddings


In [ ]:
def title_similarity():
    data["result"]=0.0
    for i in range(len(data.sentence)):
        title=stopword_remover.drop_stop_words(nltk.word_tokenize(data.loc[i,"title"].translate(translator).replace("“","").replace("’","")))
        embeddings=giveEmbeddings([data.loc[i,"sentence"],data.loc[i,"title"]])
        similarity=cosine_similarity([embeddings[0].numpy() ], [embeddings[1].numpy()])
        data.loc[i,"result"]=similarity[0][0]


In [ ]:
title_similarity()

In [ ]:
data = pd.read_csv("./data_result_added", sep='\t')
data

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,words2,sentence_length,position,result
0,1064.0,Nikah yerinizi nasıl seçersiniz?,Evlilik başvurusu nasıl yapılır?,-1,0,0,1,"['Nikah', 'yerinizi', 'nasıl', 'seçersiniz']",0.010610,0.333333,0.489865
1,1064.0,Kimler nikah şahidi seçilebilir?,Evlilik başvurusu nasıl yapılır?,-1,0,1,1,"['Kimler', 'nikah', 'şahidi', 'seçilebilir']",0.010610,0.250000,0.521200
2,1064.0,Nikahla ilgili merak ettiğiniz her şey burada…...,Evlilik başvurusu nasıl yapılır?,-1,0,0,0,"['Nikahla', 'ilgili', 'merak', 'ettiğiniz', 'h...",0.029178,0.200000,0.781842
3,1064.0,Müstakbel eşinizle birlikte yaşadığınız yerdek...,Evlilik başvurusu nasıl yapılır?,-1,1,1,0,"['Müstakbel', 'eşinizle', 'birlikte', 'yaşadığ...",0.023873,0.166667,0.657137
4,1064.0,"Ancak, ayrı illerde oturuyorsanız kendi illeri...",Evlilik başvurusu nasıl yapılır?,-1,0,1,0,"['Ancak', 'ayrı', 'illerde', 'oturuyorsanız', ...",0.031830,0.142857,0.484427
...,...,...,...,...,...,...,...,...,...,...,...
11387,1697.0,"Galatasaray, Gaziantepspor maçının hazırlıklar...",Floryada Lucas Neill rüzgarı,-1,2,0,0,"['Galatasaray', 'Gaziantepspor', 'maçının', 'h...",0.042254,0.076923,0.073290
11388,1698.0,"Başbakan Recep Tayyip Erdoğan, özel bir haber ...",Erdoğan bir dava daha kazandı,-1,3,1,3,"['Başbakan', 'Recep', 'Tayyip', 'Erdoğan', 'öz...",0.200000,1.000000,0.404898
11389,1698.0,Ankara 3.,Erdoğan bir dava daha kazandı,-1,0,0,0,"['Ankara', '3']",0.013793,0.500000,0.271632
11390,1698.0,Asliye Hukuk Mahkemesi'nde görülen davanın kar...,Erdoğan bir dava daha kazandı,-1,5,1,2,"['Asliye', 'Hukuk', 'Mahkemesinde', 'görülen',...",0.117241,0.333333,0.568497


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
import pickle
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import inspect

In [ ]:
data=data.dropna()
data.isnull().sum()
data.reset_index(drop=True, inplace=True)

In [ ]:
import random

# Initialize an empty set to store unique numbers
unique_numbers = set()

# Generate random numbers until we have 2000 unique numbers
while len(unique_numbers) < 2000:
    # Generate a random number between 0 and 2500
    random_number = random.randint(0, 2500)
    # Add the number to the set if it's not already in it
    unique_numbers.add(random_number)

# Convert the set to a list and sort it
unique_numbers_list = sorted(list(unique_numbers))

In [ ]:
X_rand_train=[]
X_rand_test=[]
for i in range(len(data)):
  if data.loc[i,"news_id"] in unique_numbers_list:
    X_rand_train.append(data.loc[i,:])
  else:
    X_rand_test.append(data.loc[i,:])

In [ ]:
random_train = pd.DataFrame (X_rand_train, columns = data.columns)
random_test = pd.DataFrame (X_rand_test, columns = data.columns)

In [ ]:
X_tr=random_train.loc[:,["news_id","sentence","title","is_contains_idiom","NER","noun_phrase","tfidf","sentence_length","position"]]
X_ts=random_test.loc[:,["news_id","sentence","title","is_contains_idiom","NER","noun_phrase","tfidf","sentence_length","position"]]
y_train=random_train.loc[:,["result"]]
y_test=random_test.loc[:,["result"]]

In [ ]:
X_train=X_tr.loc[:,["is_contains_idiom","NER","noun_phrase","tfidf","sentence_length","position"]]
X_test=X_ts.loc[:,["is_contains_idiom","NER","noun_phrase","tfidf","sentence_length","position"]]

In [ ]:
regressor = RandomForestRegressor(n_estimators = 600, random_state = 0)
regressor.fit(X_train,y_train) 

<ipython-input-14-ec162e6ddcc8>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train,y_train)


RandomForestRegressor(n_estimators=600, random_state=0)

In [ ]:
Y_pred = regressor.predict(X_test)

In [ ]:
m1=mean_squared_error(y_test,Y_pred)
m2=mean_absolute_error(y_test,Y_pred)
r2=r2_score(y_test,Y_pred, force_finite=False)
print(m1,m2,r2)

0.028971559056411973 0.13160669161260113 0.06230947780882323


In [ ]:
clf = linear_model.Lasso(alpha=0.5)
clf.fit(X_train,y_train)

Lasso(alpha=0.5)

In [ ]:
y_pred2=clf.predict(X_test)

In [ ]:
mL1=mean_squared_error(y_test,y_pred2)
mL2=mean_absolute_error(y_test,y_pred2)
r2L=r2_score(y_test,y_pred2, force_finite=False)
print(mL1,mL2,r2L)

0.03095196612669707 0.14241248082987648 -0.0017881752125712413


In [ ]:
logr  = LinearRegression()
logr.fit(X_train,y_train) 

LinearRegression()

In [ ]:
y_pred3= logr.predict(X_test)
mL1=mean_squared_error(y_test,y_pred3)
mL2=mean_absolute_error(y_test,y_pred3)
r2L=r2_score(y_test,y_pred3, force_finite=False)
print(mL1,mL2,r2L)

0.02670192204340293 0.12735477202865958 0.135768317623723


,is_contains_idiom,NER,noun_phrase,tfidf,sentence_length,position
70,-1,4,0,0,0.128205,1.000000
71,-1,2,3,0,0.076923,0.500000
72,-1,0,0,0,0.068376,0.333333
73,-1,1,1,0,0.068376,0.250000
74,-1,2,1,0,0.085470,0.200000
...,...,...,...,...,...,...
11355,-1,1,0,0,0.031414,0.076923
11356,-1,3,0,0,0.070681,0.071429
11357,-1,1,0,1,0.052356,0.066667
11358,-1,0,0,0,0.028796,0.062500


In [ ]:
X_ts.reset_index(drop=True, inplace=True)
X_ts

,news_id,sentence,title,is_contains_idiom,NER,noun_phrase,tfidf,sentence_length,position
0,1066.0,"Yahudi Ajansının yayınladığı rapora göre, geçe...",Avrupada Yahudi düşmanlığı zirvede,-1,0,0,0,0.339286,1.000000
1,1066.0,"Bu dönemin, İsrail'in Gazze işgalinden hemen s...",Avrupada Yahudi düşmanlığı zirvede,-1,2,0,0,0.178571,0.500000
2,1066.0,Örneğin Fransa'da 2009'un ilk 3 ayında 113'ü ş...,Avrupada Yahudi düşmanlığı zirvede,-1,2,0,0,0.285714,0.333333
3,1066.0,Rapora göre geçen yıl dünyada Yahudi karşıtı s...,Avrupada Yahudi düşmanlığı zirvede,-1,0,0,0,0.196429,0.250000
4,1074.0,"Merkezi İsrail'de bulunan Yahudi Ajansı, Batı ...",Yahudi düşmanlığı giderek artıyor,-1,2,0,0,0.102564,1.000000
...,...,...,...,...,...,...,...,...,...
2568,1694.0,Bir çok kiracının da avlulardaki baraka ve odu...,Romanlar 100 liraya ev sahibi olacak,-1,1,0,0,0.031414,0.076923
2569,1694.0,Neslişah ve Hatice Sultan mahallelerini kapsay...,Romanlar 100 liraya ev sahibi olacak,-1,3,0,0,0.070681,0.071429
2570,1694.0,"Ancak, bazı vatandaşlar, su, elektrik parasını...",Romanlar 100 liraya ev sahibi olacak,-1,1,0,1,0.052356,0.066667
2571,1694.0,''Sulukule Kültürü''nün yok edilmek istendiğin...,Romanlar 100 liraya ev sahibi olacak,-1,0,0,0,0.028796,0.062500


In [ ]:
def getMaxs(X,Y,Y_real):
  Y_real=Y_real.to_numpy()
  max_values = []
  real_max_values=[]
  index=-1
  real_index=-1
  curr_id=0
  max=-1
  real_max=-1
  for i in range(len(X.loc[:,"news_id"])):
    if curr_id != X.loc[i,"news_id"].astype(int):
      curr_id=X.loc[i,"news_id"].astype(int)
      if index != -1:
        max_values.append(X.loc[index,:])
        real_max_values.append(X.loc[real_index,:])
      max=-1
      index=-1
      real_max=-1
      real_index=-1
    if Y[i]>max:
      max=Y[i]
      index=i
    if Y_real[i][0]>real_max:
      real_max=Y_real[i][0]
      real_index=i
  return (max_values,real_max_values)
    


In [ ]:
(a,r_a)=getMaxs(X_ts,Y_pred,y_test)

In [ ]:
a[10]["sentence"]

'Bizim Başbakanımız, Genelkurmay Başkanımızla düzenli görüşmelerini yapıyor.'

In [ ]:
summed=0
for i in range(len(a)):
  if a[i]["sentence"]==r_a[i]["sentence"]:
    summed+=1

In [ ]:
summed/len(a)

0.2589928057553957

In [ ]:
dep_parser = DependencyParser()
parsed=dep_parser.predict(a[6]["sentence"])

KeyboardInterrupt: ignored

In [ ]:
parsed

[(1, 'Güvenlik', 2, 'nmod:poss'),
 (2, 'güçlerinin', 5, 'nsubj'),
 (3, 'olay', 4, 'nmod:poss'),
 (4, 'yerine', 5, 'obl'),
 (5, 'ulaşmaya', 6, 'ccomp'),
 (6, 'çalıştığı', 7, 'csubj'),
 (7, 'bildirildi', 0, 'root'),
 (8, '.', 7, 'punct')]

In [ ]:
dep_parser = DependencyParser()
parsed=dep_parser.predict(a[10]["sentence"])
parsed

[(1, 'Bizim', 2, 'nmod'),
 (2, 'Başbakanımız', 8, 'nsubj'),
 (3, ',', 2, 'punct'),
 (4, 'Genelkurmay', 5, 'nmod'),
 (5, 'Başkanımızla', 8, 'obl'),
 (6, 'düzenli', 7, 'amod'),
 (7, 'görüşmelerini', 8, 'obj'),
 (8, 'yapıyor', 0, 'root'),
 (9, '.', 8, 'punct')]

In [ ]:
for i in range(20):
  print(dep_parser.predict(a[i]["sentence"]))

[(1, 'Bu', 2, 'det'), (2, 'dönemin', 10, 'nsubj'), (3, ',', 2, 'punct'), (4, "İsrail'in", 8, 'nmod:poss'), (5, 'Gazze', 6, 'nmod:poss'), (6, 'işgalinden', 8, 'obl'), (7, 'hemen', 8, 'advmod'), (8, 'sonraya', 10, 'obl'), (9, 'denk', 10, 'compound:redup'), (10, 'geldiği', 11, 'csubj'), (11, 'belirtildi', 0, 'root'), (12, '.', 11, 'punct')]
[(1, 'Bu', 2, 'det'), (2, 'dönemin', 10, 'nsubj'), (3, ',', 2, 'punct'), (4, "İsrail'in", 8, 'nmod:poss'), (5, 'Gazze', 6, 'nmod:poss'), (6, 'işgalinden', 8, 'obl'), (7, 'hemen', 8, 'advmod'), (8, 'sonraya', 10, 'obl'), (9, 'denk', 10, 'compound:redup'), (10, 'geldiği', 11, 'csubj'), (11, 'belirtildi', 0, 'root'), (12, '.', 11, 'punct')]
[(1, 'Öte', 2, 'obl'), (2, 'yandan', 8, 'obl'), (3, '11.', 4, 'nummod'), (4, 'dakikada', 8, 'obl'), (5, 'Engin', 6, 'obl'), (6, "Baytar'la", 8, 'obl'), (7, 'gole', 8, 'obl'), (8, 'yaklaşan', 9, 'acl'), (9, "Trabzonspor'un", 11, 'nmod:poss'), (10, 'bir', 11, 'det'), (11, 'topu', 14, 'obj'), (12, 'da', 11, 'advmod:emph')

ValueError: ignored

In [ ]:
from google.colab import drive
import zipfile
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/nlp/zip.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
# path to jar file 
ZEMBEREK_PATH = r"tmp/zip/zemberek-full.jar"
# start jvm
startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=%s" % (ZEMBEREK_PATH),convertStrings=False)
TurkishMorphology: JClass = JClass("zemberek.morphology.TurkishMorphology")
PerceptronNer: JClass = JClass("zemberek.ner.PerceptronNer")
Paths: JClass = JClass("java.nio.file.Paths")
modelRoot = Paths.get('zip/test-model/model')
morphology = TurkishMorphology.createWithDefaults()
ner = PerceptronNer.loadModel(modelRoot, morphology)

OSError: ignored

In [ ]:
jvm.stop()

NameError: ignored

In [54]:
import pickle
with open("test3", "rb") as fp:   # Unpickling
  b = pickle.load(fp)

In [61]:
b[0]


[(1, 'Geçtiğimiz', 2, 'acl'),
 (2, 'günlerde', 11, 'obl'),
 (3, 'sarı-lacivertli', 4, 'amod'),
 (4, 'yönetim', 11, 'nsubj'),
 (5, ',', 11, 'punct'),
 (6, 'yıldız', 7, 'amod'),
 (7, 'oyuncuya', 8, 'obl'),
 (8, 'sormadan', 11, 'advcl'),
 (9, 'opsiyon', 10, 'nmod:poss'),
 (10, 'hakkını', 11, 'obj'),
 (11, 'kullanmış', 0, 'root'),
 (12, 've', 16, 'cc'),
 (13, 'sözleşmeyi', 16, 'obj'),
 (14, '1', 15, 'nummod'),
 (15, 'yıllık', 16, 'amod'),
 (16, 'uzatmıştı', 11, 'conj'),
 (17, '.', 11, 'punct')]

In [47]:
b[0]

[(1, 'Geçtiğimiz', 2, 'acl'),
 (2, 'günlerde', 11, 'obl'),
 (3, 'sarı-lacivertli', 4, 'amod'),
 (4, 'yönetim', 11, 'nsubj'),
 (5, ',', 11, 'punct'),
 (6, 'yıldız', 7, 'amod'),
 (7, 'oyuncuya', 8, 'obl'),
 (8, 'sormadan', 11, 'advcl'),
 (9, 'opsiyon', 10, 'nmod:poss'),
 (10, 'hakkını', 11, 'obj'),
 (11, 'kullanmış', 0, 'root'),
 (12, 've', 16, 'cc'),
 (13, 'sözleşmeyi', 16, 'obj'),
 (14, '1', 15, 'nummod'),
 (15, 'yıllık', 16, 'amod'),
 (16, 'uzatmıştı', 11, 'conj'),
 (17, '.', 11, 'punct')]

In [152]:
titles=[]
for deps_pars in b:
  root=0
  zero=0
  title = set()
  indexes=set()
  deps_pars
  if type(deps_pars[0])==str:
    titles.append("şok şok şok")
    continue
  if len(deps_pars)<14:
    titles.append("")
    continue
  for j in range(len(deps_pars)):
    i=deps_pars[j]
    if i[2]==0:
      zero=i
      indexes.add(i[0])
      title.add(i)
    if i[3]=="root":
      root=i
      indexes.add(i[0])
      title.add(i)
  
  for j in range(len(deps_pars)):
    i=deps_pars[j]
    if (root!=0 and i[0]+1==root[0]) or (zero!=0 and  i[0]+1==zero[0]):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

    if  (root!=0 and (i[0]-1==root[0] or i[0]+1==root[0]) and i[3]=="conj") or (zero!=0 and (i[0]-1==zero[0] or i[0]+1==zero[0]) and i[3]=="conj"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

    if ((root!=0 and i[2]==root[0] ) or (zero!=0 and i[2]==zero[0]) ) and (i[3]=="nsubj" or i[3]=="compound" or i[3]=="flat"   ):
        if i[2]!="punct":
          title.add(i)
          indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])
  if(len(title)==0 or title==None):
    title="şok şok şok"
  titles.append(title)




In [150]:
basliks=[]
for title in titles:
  baslik=""
  if type(title)==str:
    basliks.append(title)
    continue
  for kelime in title:
    baslik +=" "+kelime[1]
  basliks.append(baslik)



In [153]:
titles

[{(2, 'günlerde', 11, 'obl'),
  (3, 'sarı-lacivertli', 4, 'amod'),
  (4, 'yönetim', 11, 'nsubj'),
  (9, 'opsiyon', 10, 'nmod:poss'),
  (10, 'hakkını', 11, 'obj'),
  (11, 'kullanmış', 0, 'root')},
 {(21, "Komutanlığı'ndaki", 23, 'amod'),
  (22, 'cuntacı', 23, 'amod'),
  (23, 'yapıyla', 25, 'obl'),
  (25, 'gözler', 26, 'nmod:poss'),
  (26, 'önüne', 27, 'obl'),
  (27, 'serdi', 0, 'root')},
 {(8, 'hissederler', 10, 'nmod:poss'),
  (9, '"', 8, 'nmod:poss'),
  (10, 'sözü', 14, 'nsubj'),
  (12, 'bu', 13, 'det'),
  (13, 'iddiaları', 14, 'obj'),
  (14, 'güçlendirdi', 0, 'root')},
 {(1, 'Trendeydik', 0, 'root')},
 {(24, 'hayatını', 25, 'obj'), (25, 'kaybetti', 0, 'root')},
 {(34, 'cinsel', 35, 'amod'),
  (35, 'istismara', 38, 'obl'),
  (38, 'evliliğe', 29, 'obl'),
  (39, 'zorlandı', 0, 'root')},
 {(14, 'incelemede', 29, 'obl'),
  (23, '50', 24, 'nummod'),
  (24, 'karton', 26, 'amod'),
  (25, 'gümrük', 26, 'nmod:poss'),
  (26, 'kaçağı', 27, 'nmod:poss'),
  (27, 'sigara', 28, 'nsubj'),
  (28, 'ele

In [154]:
basliks

[' yönetim hakkını opsiyon günlerde sarı-lacivertli kullanmış',
 " gözler yapıyla Komutanlığı'ndaki cuntacı önüne serdi",
 ' " iddiaları sözü bu hissederler güçlendirdi',
 ' Trendeydik',
 ' kaybetti hayatını',
 ' cinsel zorlandı istismara evliliğe',
 ' 50 geçirildi incelemede sigara ele karton kaçağı gümrük',
 ' ÜSTÜNE YAZDI',
 ' çıkmış ortaya olduğu',
 " toplamının gelirlerinden ek dahil gösterge yan ödeme profesör her ek 800'ünü kadrosuna geçemeyecek",
 ' ilgili ifade gerektiğini kararına sağlaması',
 ' Sylvain hale bir hassas gelecektir Mimoun bu nokta işlevsel',
 '',
 ' yarın bildirildi dakika bugün 10.00 6 yerel duyulacağı saatle 14.00',
 ' yönetiminde endüstrisinin mali ilkelerinden diye kurumsal yönetim fubol konuştu faydalanılabileceğine yapılarına Türk',
 ' yola yan TIR devrilmedi',
 '',
 ' gösteriyor olarak',
 ' bakım genç kalım kız mücadelesi yoğun veriyor ölüm ünitesinde',
 ' giderilmelidir kurumlarda 4-C arada mağduriyetleri Bu',
 " Albay Varımlı'nın İddianamesi'nde Poyraz

In [157]:
from summa import summarizer
print(summarizer.summarize(text))

In [ ]:
data = pd.read_csv("./data_result_added", sep='\t')
data